In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
model = TfidfVectorizer(stop_words='english')

In [4]:
df = pd.read_csv("data/netflix_titles.csv")
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...


In [5]:
df['description'] = df['description'].fillna('')

In [6]:
feature_matrix = model.fit_transform(df['description'])

In [7]:
feature_matrix

<7787x17905 sparse matrix of type '<class 'numpy.float64'>'
	with 107187 stored elements in Compressed Sparse Row format>

In [8]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(feature_matrix, feature_matrix)

In [9]:
indices = pd.Series(df.index,index=df['title']).drop_duplicates()

In [10]:
def get_recommendations(title):
    idx = indices[title]
    #вычисляем попарные коэффициенты косинусной близости
    scores = list(enumerate(cosine_sim[idx]))
    #сортируем фильмы на основании коэффициентов косинусной близости по убыванию
    scores = sorted(scores, key=lambda x: x[1], reverse=True)
    #выбираем десять наибольших значений косинусной близости; нулевую не берём, т. к. это тот же фильм
    scores =   scores[1:11]
    #забираем индексы
    ind_movie = [i[0] for i in scores]
    #возвращаем названия по индексам
    return df['title'].iloc[ind_movie]

In [11]:
get_recommendations('Balto')

709                Balto 2: Wolf Quest
7446                           Vroomiz
1338    Chilling Adventures of Sabrina
7388                          Vampires
1770                          Dinotrux
2767                     Hold the Dark
5540                 Shanghai Fortress
4041                             Mercy
2582                       Half & Half
1365        Christmas in the Heartland
Name: title, dtype: object

In [12]:
from surprise import Dataset
from surprise import Reader
from surprise.dataset import BUILTIN_DATASETS #с помощью данного объекта мы можем использовать встроенные датасеты

data = Dataset.load_from_file(
    "u.data.txt",
    reader=Reader(line_format="user item rating timestamp", sep="\t"),
)

ModuleNotFoundError: No module named 'surprise'

In [ ]:
df['rating'].value_counts

In [13]:
X_train, X_test = surprise.model_selection.train_test_split(random_state = 13)

NameError: name 'surprise' is not defined

In [16]:
from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k, recall_at_k    

ModuleNotFoundError: No module named 'lightfm'

# 5. Современные методы: глубокое обучение

In [42]:
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from tensorflow.keras.models import Model

In [25]:
from sklearn.model_selection import train_test_split

In [26]:
df = pd.read_csv('data/ratings.csv')

In [51]:
train, test = train_test_split(df, test_size=0.2, random_state = 13)

In [52]:
train.shape

(785404, 3)

In [33]:
df

,book_id,user_id,rating
0,1,314,5
1,1,439,3
2,1,588,5
3,1,1169,4
4,1,1185,4
...,...,...,...
981751,10000,48386,5
981752,10000,49007,4
981753,10000,49383,5
981754,10000,50124,5


In [38]:
n_books = df['book_id'].unique().shape
n_books

(10000,)

In [39]:
n_users = df['user_id'].unique().shape
n_users

(53424,)

In [40]:
n_books = df["book_id"].nunique()
print(n_books)
n_users = df["user_id"].nunique()
print(n_users)

10000
53424


In [43]:
book_input = Input(shape=[1], name="Book-Input")
book_embedding = Embedding(n_books+1, 5, name="Book-Embedding")(book_input)
book_vec = Flatten(name="Flatten-Books")(book_embedding)

In [45]:
user_input = Input(shape=[1], name="User-Input")
user_embedding = Embedding(n_users+1, 5, name="User-Embedding")(user_input)
user_vec = Flatten(name="Flatten-Users")(user_embedding)

In [46]:
conc = Concatenate()([book_vec, user_vec])

In [47]:
fc1 = Dense(128, activation='relu')(conc)
fc2 = Dense(32, activation='relu')(fc1)
out = Dense(1)(fc2)

In [48]:
model2 = Model([user_input, book_input], out)

In [49]:
model2.compile(optimizer = 'adam',loss =  'mean_squared_error')

In [53]:
history = model2.fit([train.user_id, train.book_id], train.rating, epochs=5, verbose=1)

Epoch 1/5
24544/24544 [==============================] - 155s 6ms/step - loss: 0.7992
Epoch 2/5
24544/24544 [==============================] - 174s 7ms/step - loss: 0.6900
Epoch 3/5
24544/24544 [==============================] - 166s 7ms/step - loss: 0.6598
Epoch 4/5
24544/24544 [==============================] - 159s 6ms/step - loss: 0.6361
Epoch 5/5
24544/24544 [==============================] - 161s 7ms/step - loss: 0.6143


In [54]:
model2.evaluate([test.user_id, test.book_id], test.rating)

6136/6136 [==============================] - 7s 1ms/step - loss: 0.6995


0.6994650959968567

In [55]:
fc1 = Dense(128, activation='relu')(conc)
fc2 = Dense(32, activation='relu')(fc1)
fc3 = Dense(8, activation='relu')(fc2)
out = Dense(1)(fc3)

model2 = Model([user_input, book_input], out)
model2.compile('adam', 'mean_squared_error')
result = model2.fit([train.user_id, train.book_id], train.rating, epochs=10, verbose=1)
model2.evaluate([test.user_id, test.book_id], test.rating)

Epoch 1/10
24544/24544 [==============================] - 160s 6ms/step - loss: 0.6229
Epoch 2/10
24544/24544 [==============================] - 170s 7ms/step - loss: 0.5813
Epoch 3/10
24544/24544 [==============================] - 172s 7ms/step - loss: 0.5624
Epoch 4/10
24544/24544 [==============================] - 172s 7ms/step - loss: 0.5458
Epoch 5/10
24544/24544 [==============================] - 167s 7ms/step - loss: 0.5310
Epoch 6/10
24544/24544 [==============================] - 180s 7ms/step - loss: 0.5190
Epoch 7/10
24544/24544 [==============================] - 177s 7ms/step - loss: 0.5089
Epoch 8/10
24544/24544 [==============================] - 183s 7ms/step - loss: 0.5000
Epoch 9/10
24544/24544 [==============================] - 155s 6ms/step - loss: 0.4926
Epoch 10/10
6136/6136 [==============================] - 8s 1ms/step - loss: 0.7705


0.7705029845237732